In [1]:
import pandas as pd
import os
from datetime import datetime

In [9]:
def df_processing(file_name):
    df = pd.read_csv(file_name)
    print("Processing {}...".format(file_name))
    print("len avant: ", len(df))
    df = df.loc[df.date_experience!="Temps d'utilisation du logiciel : Essai gratuit"]
    df = df.loc[df.date_experience!="Temps d'utilisation du logiciel : Non fourni"]
    df = df.reset_index(drop=True)
    print("len apres: ", len(df))

    df = df.rename(columns={"date_experience": "date_experience_del", "timestamp": "timestamp_del"})

    data_experience = {"Temps d'utilisation du logiciel : 6 à 12 mois":9,
                       "Temps d'utilisation du logiciel : plus de deux ans":24,
                       "Temps d'utilisation du logiciel : plus d'un an":12,
                       "Temps d'utilisation du logiciel : 1 à 5 mois":3}
    df["date_experience"] = df.apply(lambda row: data_experience[row["date_experience_del"]], axis=1)
    df = df.drop(["date_experience_del"], axis=1)

    timestamp = {'il y a 2 ans':24, 'il y a 3 mois':3, 'il y a 5 mois':5, 'il y a 4 mois':4,
                 'il y a 2 mois':2, 'il y a 10 mois':10, 'il y a 7 ans':7*12, 'il y a 3 ans':3*12,
                 'il y a 6 ans':6*12, 'il y a 12 mois':12, 'il y a 4 ans':4*12,
                 'l’année dernière':6*12, 'le mois dernier':1/2, 'il y a 5 ans':5*12,
                 'il y a 9 mois':9, 'il y a 4 semaines':28/30, 'il y a 3 semaines':21/30,
                 'il y a 11 mois':11, 'il y a 8 mois':8, 'il y a 6 mois':6, 'il y a 8 ans':8*12,
                 'il y a 2 semaines':14/30, 'la semaine dernière':3.5/30, 'il y a 7 mois':7,
                 'il y a 4 jours':4/30, 'hier':1/30, 'il y a 6 jours':6/30, 'il y a 5 jours':5/30,
                 'il y a 3 jours':3/30, 'il y a 2 jours':2/30, 'il y a 18 heures':(18/24)/30}
    dt = datetime.now().timestamp()-os.path.getctime(file_name)
    dt = dt/(60*60) # heures
    dt = dt/24 # jours
    dt = dt/30 # mois
    df["timestamp"] = df.apply(lambda row: dt+timestamp[row["timestamp_del"]], axis=1)
    df = df.drop(["timestamp_del"], axis=1)

    df = df[["user_name", "review_score", "title", "comment", "date_experience", "timestamp"]]
    
    return df

In [10]:
path = "../../data_csv/"
files = []
for el in os.listdir(path):
    if (el[0].isdigit())and(el[-4:]==".csv"):
        files.append(path + el)
files

['../../data_csv/75_100_62%.csv',
 '../../data_csv/100_125.csv',
 '../../data_csv/200_225.csv',
 '../../data_csv/150_175.csv',
 '../../data_csv/125_150.csv',
 '../../data_csv/175_200.csv',
 '../../data_csv/0_25.csv',
 '../../data_csv/25_75.csv']

In [11]:
df2 = df_processing(files[0])
for file in files[1:]:
    df2 = pd.concat([df2, df_processing(file)], ignore_index=True)

Processing ../../data_csv/75_100_62%.csv...
len avant:  4165
len apres:  3978
Processing ../../data_csv/100_125.csv...
len avant:  6774
len apres:  6719
Processing ../../data_csv/200_225.csv...
len avant:  6530
len apres:  6191
Processing ../../data_csv/150_175.csv...
len avant:  4816
len apres:  4717
Processing ../../data_csv/125_150.csv...
len avant:  3022
len apres:  2969
Processing ../../data_csv/175_200.csv...
len avant:  3310
len apres:  3253
Processing ../../data_csv/0_25.csv...
len avant:  459
len apres:  436
Processing ../../data_csv/25_75.csv...
len avant:  8862
len apres:  8684


In [12]:
print(len(df2))
df3 = df2.dropna().reset_index(drop=True)
print(len(df3))

36947
36189


In [14]:
df3.head(10)

,user_name,review_score,title,comment,date_experience,timestamp
0,Marion,"4,0",Agréable pour la gestion,Un outil très pratique pour la gestion en comp...,12,5.000458
1,Utilisateur vérifié,"4,0",Utilisateur_Trello,Trello a permis tous les membres de notre équi...,24,72.000458
2,Utilisateur vérifié,"5,0",Outil de gestion projet simple et intuitif,Très bonne expérience globale sur cet outil. J...,12,72.000458
3,Utilisateur vérifié,"4,0",Trello - Une liste de taches virtuelle,"Selon mon experience, j'ai pu gérer mes projet...",12,72.000458
4,Céline,"5,0",L’incontournable logiciel de gestion de projet,Trello est un tableau d’affichage d’idée très ...,24,36.000458
5,Alexandro,"5,0",Mon outil d'organisation préféré,Après n'avoir pas avoir été très productif dan...,12,36.000458
6,Utilisateur vérifié,"5,0",Je recommande Trello pour tous,Trello me fait gagner beaucoup de temps dans m...,12,10.000458
7,Sandrine,"5,0",Un outil indispensable sur Mac lorsque l'on gè...,C'est un outil dont je ne peux plus me passer ...,24,24.000458
8,Hugo,"4,0",Bon dans l'ensemble,Trello me permet d'organiser l'ensemble de mes...,24,36.000458
9,Julien,"5,0",à propos de trello,Nous avons cherché longtemps un soft en frança...,24,48.000458


In [17]:
filename = "capterra.csv"
df3.to_csv(path+filename, index=False)